In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import MeCab
m = MeCab.Tagger()

urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])

# 레이블 만들기 (평점이 4~5인 리유에는 레이블 1을, 평점이 1~2인 리뷰는 레이블 0을 부여)
total_data['label'] = np.select([total_data.ratings > 3], [1], default=0)

# 각 열에 대해서 중복을 제외한 샘플의 수를 카운트
total_data['ratings'].nunique(), total_data['reviews'].nunique(), total_data['label'].nunique()

# reviews 열에서 중복인 내용이 있다면 중복 제거
total_data.drop_duplicates(subset=['reviews'], inplace=True)

# 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)


# 데이터 정제하기
# 트레인 데이터 한글과 공백을 제외하고 모두 제거
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['reviews'].replace('', np.nan, inplace=True)


# 테스트 데이터
test_data.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거


# 토큰화
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")


stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']


train_data['tokenized'] = train_data['reviews'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


test_data['tokenized'] = test_data['reviews'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])


negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

# 부정 리뷰에서 주로 등장하는 것들
negative_word_count = Counter(negative_words)
# 긍정 리뷰에서 주로 등장하는 것들
positive_word_count = Counter(positive_words)


X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)


threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value


vocab_size = total_cnt - rare_cnt + 2

tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1


max_len = 80
below_threshold_len(max_len, X_train)


X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)



from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)



loaded_model = load_model('best_model.h5')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xeb in position 529514: invalid continuation byte

In [3]:
# 감성 분석
new_review = pd.read_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\치아미백 전처리.csv')
new_review = new_review.dropna(axis=0)
new_review = new_review.drop_duplicates()

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정^^ 리뷰입니다.".format(score * 100))
    return ("{:.2f} 1".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정!! 리뷰입니다.".format((1 - score) * 100))
    return ("{:.2f} 2".format((1 - score) * 100))
  
new_review['감성분석'] = new_review['내용'].apply(lambda x:sentiment_predict(x))
new_review.to_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\치아미백 감성분석.csv', encoding = 'utf-8-sig')








new_review = pd.read_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\치주염 전처리.csv')
new_review = new_review.dropna(axis=0)
new_review = new_review.drop_duplicates()

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정^^ 리뷰입니다.".format(score * 100))
    return ("{:.2f} 1".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정!! 리뷰입니다.".format((1 - score) * 100))
    return ("{:.2f} 2".format((1 - score) * 100))
  
new_review['감성분석'] = new_review['내용'].apply(lambda x:sentiment_predict(x))
new_review.to_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\치주염 감성분석.csv', encoding = 'utf-8-sig')







new_review = pd.read_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\구내염 전처리.csv')
new_review = new_review.dropna(axis=0)
new_review = new_review.drop_duplicates()

def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords]
  encoded = tokenizer.texts_to_sequences([new_sentence])
  pad_new = pad_sequences(encoded, maxlen = max_len)

  score = float(loaded_model.predict(pad_new))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정^^ 리뷰입니다.".format(score * 100))
    return ("{:.2f} 1".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정!! 리뷰입니다.".format((1 - score) * 100))
    return ("{:.2f} 2".format((1 - score) * 100))
  
new_review['감성분석'] = new_review['내용'].apply(lambda x:sentiment_predict(x))
new_review.to_csv(r'C:\Users\82108\Coding\MZ-trend\트위터\클린디 트위터 로우 데이터\구내염 감성분석.csv', encoding = 'utf-8-sig')

70.93% 확률로 부정!! 리뷰입니다.
91.47% 확률로 부정!! 리뷰입니다.
95.30% 확률로 긍정^^ 리뷰입니다.
77.50% 확률로 부정!! 리뷰입니다.
69.27% 확률로 부정!! 리뷰입니다.
71.36% 확률로 부정!! 리뷰입니다.
94.86% 확률로 부정!! 리뷰입니다.
99.30% 확률로 긍정^^ 리뷰입니다.
75.31% 확률로 부정!! 리뷰입니다.
59.20% 확률로 부정!! 리뷰입니다.
88.02% 확률로 긍정^^ 리뷰입니다.
99.28% 확률로 부정!! 리뷰입니다.
59.41% 확률로 부정!! 리뷰입니다.
60.17% 확률로 부정!! 리뷰입니다.
92.76% 확률로 부정!! 리뷰입니다.
75.31% 확률로 부정!! 리뷰입니다.
85.09% 확률로 부정!! 리뷰입니다.
89.71% 확률로 부정!! 리뷰입니다.
90.89% 확률로 긍정^^ 리뷰입니다.
91.16% 확률로 부정!! 리뷰입니다.
87.21% 확률로 부정!! 리뷰입니다.
74.68% 확률로 부정!! 리뷰입니다.
64.28% 확률로 부정!! 리뷰입니다.
79.76% 확률로 부정!! 리뷰입니다.
54.01% 확률로 부정!! 리뷰입니다.
60.70% 확률로 부정!! 리뷰입니다.
68.03% 확률로 부정!! 리뷰입니다.
75.31% 확률로 부정!! 리뷰입니다.
53.73% 확률로 부정!! 리뷰입니다.
55.35% 확률로 부정!! 리뷰입니다.
75.31% 확률로 부정!! 리뷰입니다.
52.84% 확률로 긍정^^ 리뷰입니다.
76.22% 확률로 부정!! 리뷰입니다.
67.67% 확률로 긍정^^ 리뷰입니다.
67.67% 확률로 긍정^^ 리뷰입니다.
65.93% 확률로 부정!! 리뷰입니다.
74.70% 확률로 부정!! 리뷰입니다.
79.83% 확률로 부정!! 리뷰입니다.
97.41% 확률로 긍정^^ 리뷰입니다.
57.82% 확률로 부정!! 리뷰입니다.
57.82% 확률로 부정!! 리뷰입니다.
67.45% 확률로 부정!! 리뷰입니다.
92.21% 확률로 부정!! 리뷰입니다.
57.58% 확률로 